link:  https://bit.ly/mario-live25

Usar uma rede já treinada (tranfer learning) para resolver problema de NLP: Classificar produtos rotulando-os com tags.

Isso é um problema muito comun em sites de e-commerces. Classifica automaticamente os produtos.

Vamos usar o modelo dofacebook chamado "Roberta" (bert). Ele tenta criar uma representaçâo vetorial de uma linguagem. Esse modelo é treinado em parte de palavras, então, podemos usá-lo em qualquer lugar

In [ ]:
# biblioteca que tem modelos pré-treinados
!pip install transformers

     |████████████████████████████████| 573kB 4.8MB/s 
     |████████████████████████████████| 1.0MB 12.1MB/s 
     |████████████████████████████████| 3.7MB 33.2MB/s 
     |████████████████████████████████| 890kB 59.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=3aa49a72ab1728450a56cc2b6b2de35cbce5cf0fc1fc43bcc7fdee2a624a7691
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import transformers
import torch.utils.data as tdata
import torch.optim as optim

import tqdm # serve para colocar uma barra de loading e qualquer loop

In [ ]:
train = pd.read_csv("Hackathon_Base_Treino.csv")
train.head()

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA
2,AGUA MIN SCHIN S/GAS 500ML,SEM GÁS,ÁGUA MINERAL
3,FITA DUPLA FACE C/SUPORTE SCOTCH,FITA ADESIVA,PAPELARIA
4,MASSA PIZZA ROMANHA OREGANO PCT 160G,PIZZA REGULAR,MASSA FRESCA


In [ ]:
# 89% das categorias tem mais de 10 itens
(train['CATEGORIA'].value_counts() > 10).mean()

0.8945783132530121

In [ ]:
# Lista de categorias
train['CATEGORIA'].value_counts()

VINHO                          789
BOMBONIERE                     672
BISCOITO                       575
UTENSÍLIOS DE INOX/ALUMÍNIO    525
CALÇADO ADULTO                 510
                              ... 
EMPADA                           6
BOLINHO                          5
FRANGO                           4
ESPETINHO                        4
FRUTA DESIDRATADA                3
Name: CATEGORIA, Length: 332, dtype: int64

In [ ]:
# Tranformar de String para número
from sklearn.preprocessing import LabelEncoder

cats_encoder = LabelEncoder()
train['encoded_labels'] = cats_encoder.fit_transform(train['CATEGORIA'])

In [ ]:
train.head()

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,encoded_labels
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,85
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,124
2,AGUA MIN SCHIN S/GAS 500ML,SEM GÁS,ÁGUA MINERAL,327
3,FITA DUPLA FACE C/SUPORTE SCOTCH,FITA ADESIVA,PAPELARIA,230
4,MASSA PIZZA ROMANHA OREGANO PCT 160G,PIZZA REGULAR,MASSA FRESCA,197


In [ ]:
# Colocar na CPU
device = torch.device("cuda:0")

# 
roberta_weights = 'roberta-base'
roberta_model = transformers.RobertaModel.from_pretrained(roberta_weights).to(device) # o modelo
roberta_token = transformers.RobertaTokenizer.from_pretrained(roberta_weights) # converter as trings em token para a NLP

# unsqueeze(0) => para passar para o pytorch, o torch precisa mudar uma dimensâo a mais (o batch_size)
# Vamos converter X em tokens (as descrições dos produtos)
tokenized = [torch.tensor(roberta_token.encode(x)).unsqueeze(0).to(device) for x in train['DESCRIÇÃO PARCEIRO']]


In [ ]:
# TOKEN: Assim, as descrições vão ser representadas pelo seguinte arraynumerico
tokenized[0]

tensor([[    0,   221,  2336,  3847, 30497,   468,  2068,   591, 37420,   112,
             6, 31866,   530,   534,  3326,  9309,    73, 10237,  3293,     2]],
       device='cuda:0')

In [ ]:
# EMBEDDINGS: De token vamos converter num arrya maior e mais esparso
#   Ele vai fazer as relações entre a semâmtica das palavras
embeddings = []

roberta_model.eval() # trava o modelo para ele nao treinar quando fizer o feedward
with torch.no_grad(): # nao salvar o gradiente e nao fazer nada de nada ...
  for x in tqdm.notebook.tqdm(tokenized):
    # vou fazer o processo de embedding  para meu X, vou pegalo e mandalo para um numpy array
    embeddings.append(roberta_model(x)[1].cpu().numpy())

In [ ]:
# Agora para cada X tem tamanho de 768
embeddings[0].shape

(1, 768)

In [ ]:
# o squeeze vai tirar essa dimensao amais do pytorch
# Vamos ter então o nosso X embeddizado
embeddings_numpy = np.array(embeddings).squeeze()

In [ ]:
embeddings_numpy.shape

(22009, 768)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Dividir em train/test
Xtrain, Xval, ytrain, yval = train_test_split(embeddings_numpy, train['encoded_labels'] , train_size=0.5, random_state=0)

print(Xtrain.shape, Xval.shape, ytrain.shape, yval.shape)

(11004, 768) (11005, 768) (11004,) (11005,)


In [ ]:
# Y : Temos 331 classes (multi-classe) e só podemos ter somente uma única classe
ytrain.unique().shape

(331,)

In [ ]:
from sklearn.metrics import f1_score

Abordagem um-contra-todos:

**VAMOS TREINAR O MODELO CLASE POR CLASSE**:
Se sâo N classe, vamos fazer um `for` de N, onde para cada N faremos o treinamento da nossa NN com um problema de classificaçâo Binária, onde a classe escolhida terá label 1 e todas as outras 0.

In [ ]:
# Array vazio para guardar as nossa previsões
predictions = np.zeros((Xval.shape[0], 332))

# Para cada classe ...
for class_ in sorted(train['encoded_labels'].unique()):
  # Vamos criar uma mask de Boolean para que:
  #   CADA 'FOR' FICA PARA TREINAR COM UMA ÚNICA CLASSE
  #   Não há um paper para explicar essa abordagem mas Mário disse que funciona
  mask_train = ytrain==class_
  mask_val = yval==class_

  # Se a classe tiver poucos dados (menos de 10) entâo não vou treinar pra isso
  if mask_train.sum() + mask_val.sum() < 10:
    continue

  # Converto mesmo de Bool para Int (classe)
  ytrain_ = mask_train.astype(int)
  yval_ = mask_val.astype(int)

  # class_weight = 'auto' : vai corrigir o desbalancemento por usar 'um-contra-todos'
  #   Vai dar uma punição quando houver desbalancimento de classe
  model = LogisticRegression(class_weight='auto')
  model.fit(Xtrain, ytrain_)

  # Prevê se pertence ou nâo a classe
  p = model.predict_proba(Xval)[:,1] # ele retorna duas prob, uma para '0' e outra para '1'

  predictions[:, class_] = p

  # threshold = ponto de corte
  # Eu quero que o ponto de corte seja a prediçâo dos 1% melhor classificados como 1
  threshold = np.percentile(p, 99)

  # se a proba for acima do threshodl, 1, else, 0
  p_cut = (p > threshold).astype(int)

  print("Class = {} | Num exemplos positivos | train = {} | val = {} | F1 = {} | p-avg = {}\n".format(class_, ytrain_.sum(), yval_.sum(), f1_score(yval_, p_cut), p.mean()))
  #break

Class = 0 | Num exemplos | train = 41 | val = 35 | F1 = 0.4109589041095891 | p-avg = 0.0037443804220666076

Class = 1 | Num exemplos | train = 18 | val = 28 | F1 = 0.043165467625899276 | p-avg = 0.0016374352061380543

Class = 2 | Num exemplos | train = 32 | val = 29 | F1 = 0.11428571428571428 | p-avg = 0.0029124203511822874

Class = 4 | Num exemplos | train = 20 | val = 29 | F1 = 0.07142857142857142 | p-avg = 0.0018171148641719364

Class = 5 | Num exemplos | train = 26 | val = 18 | F1 = 0.06201550387596899 | p-avg = 0.002366189401969137

Class = 6 | Num exemplos | train = 49 | val = 67 | F1 = 0.39325842696629215 | p-avg = 0.004458379283504561

Class = 7 | Num exemplos | train = 46 | val = 45 | F1 = 0.16666666666666666 | p-avg = 0.004192869708728135

Class = 8 | Num exemplos | train = 13 | val = 16 | F1 = 0.14173228346456693 | p-avg = 0.0011812913233086493

Class = 9 | Num exemplos | train = 30 | val = 25 | F1 = 0.20588235294117643 | p-avg = 0.002727213457357603

Class = 10 | Num exempl

In [ ]:
# ideias para prever tudo?
predictions

array([[0.00469215, 0.00165317, 0.00319488, ..., 0.0010095 , 0.00115814,
        0.00080193],
       [0.00384491, 0.00155513, 0.00314089, ..., 0.00102776, 0.00114604,
        0.00078925],
       [0.00309404, 0.00153687, 0.00253965, ..., 0.00099568, 0.00119953,
        0.00081307],
       ...,
       [0.00319281, 0.00156974, 0.00282051, ..., 0.00095723, 0.00121586,
        0.0008319 ],
       [0.00591291, 0.00144829, 0.00296864, ..., 0.00104783, 0.00141929,
        0.00078304],
       [0.00378314, 0.0016953 , 0.00302619, ..., 0.0009483 , 0.00112196,
        0.00085317]])

In [ ]:
# Uma primeira abordagem seria pegar aquele que tem maior proba
p_argmax = predictions.argmax(axis=1)
# Mas, essa proba foi calculada somente em uma classe, entô, vamos fazer UM PÓS-PROCESSAMENTO

In [ ]:
# vai calcular independente da classe
f1_score(yval, p_argmax, average='micro')

0.16492503407542025

In [ ]:
# rankdata do scipy: ela faz o ranking das coisas
#   ESSE RANKEAMENTO É DIFERNETE: QUANTO MAIOR O VALOR DO RANKING MAIOR É O VALOR DA PROBA
#     Entâo, ranking 9999° > 1° (último)
from scipy.stats import rankdata

p_top_rank = np.zeros(predictions.shape)

# para cada classe
for class_ in range(predictions.shape[1]):
  # você rankeia as previsões POR MODELO (ENTENDA É POR MODELO)
  # Isso vai permitir que você busque as previsões melhores classificadas por modelo que é DIFERETE DA MAIOR PROBA ENTRE OS MODELOS
  p_top_rank[:,class_] = rankdata(predictions[:, class_])

p_top_rank = p_top_rank.argmax(axis=1)

In [ ]:
# Em vez de termos o array de probas, agora temos o de ranking
#   Ex: o primeiro elemetno [9528] quer dizer que essa proba está na posiçâo 
#       9528° 
p_top_rank

array([[ 9528.,  6388.,  9999., ...,  6707.,  4695.,  3202.],
       [ 7038.,  2898.,  9498., ...,  8404.,  3961.,  1513.],
       [ 2620.,  2236.,   302., ...,  5185.,  6967.,  4971.],
       ...,
       [ 3224.,  3470.,  3810., ...,  1563.,  7730.,  7728.],
       [10707.,   283.,  6818., ...,  9734., 10965.,   882.],
       [ 6737.,  7688.,  7961., ...,  1058.,  2681.,  9885.]])

In [ ]:
# vai melhorar? qual ganha?
f1_score(yval,p_top_rank, average='micro')

0.4023625624716038

Sim

Com um mesmo modelo, fazendo um pós-processamento, saiumos de um f1_score de 0.16 para 0.40 com algo muito simples.

In [ ]:
# threshold optimization by class
# vs bag of words? tfidf, ensemble